<a href="https://colab.research.google.com/github/EAFIT-BI/Supervised-Learning-2024-II/blob/main/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#cargamos librerías

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image

In [3]:
# conectamos el sistema de archivos del drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# cargamos el archivo de etiquetas
labels_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Aprendizaje Supervisado 2024 II/fotos/informacion.xlsx',
                          header = None)

In [ ]:
#visualizamos el archivo
labels_df.head()

In [ ]:
#separamos los nombres de los archivos de las etiquetas
filenames = labels_df[0].values
labels = labels_df[1].values

In [ ]:
from sklearn.preprocessing import LabelEncoder

#codificamos las etiquetas
#instanciamos el codificador
label_encoder = LabelEncoder()
#aplicamos la codificación en las etiquetas
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
#aplicamos el one_hot_encoding para tener las salidas
#requeridas por el softmax
labels_one_hot = tf.keras.utils.to_categorical(labels_encoded, num_classes=2)

In [20]:
#ajustamos la resolución de las imágenes para homogenizar y
#bajar la carga computacional del procesamiento

#definimos el tamaño
img_size = (128,128)

In [17]:
#definimos una función de carga y procesamiento de las imágenes

def load_and_preprocess_image(filename, img_size):
  # abrimos y cargamos cada imagen en la variable img
  img = Image.open(filename)
  # cambiamos el tamaño de la imagen
  img = img.resize(img_size)
  # ponemos los valores de intensidad entre 0 y 1
  img = np.array(img).astype('float32') / 255
  if img.shape[-1] == 4: #comprobamos si existe canal alfa
    img = img[...,:3]
  return img

In [21]:
#cargamos las imágenes desde nuestro sistema de archivos

#agregamos la extensión a cada nombre de la imagen
filenames_with_extension = [fname + ".jpg" for fname in filenames]
#difinimos el directorio con las imágenes
image_dir = '/content/drive/MyDrive/Colab Notebooks/Aprendizaje Supervisado 2024 II/fotos/fotos/'
#cargamos las imágenes
images = np.array([load_and_preprocess_image(os.path.join(image_dir,
                                                          fname),img_size) for fname in filenames_with_extension])

In [22]:
images.shape

(92, 128, 128, 3)

In [ ]:
#si queremos las imágenes del conjunto de datos (no las matrices numéricas)
import matplotlib.pyplot as plt
plt.imshow(images[91,:,:,:]);

In [26]:
#dividimos nuestro conjunto de imágenes en train y test

train_images, test_images, train_labels, test_labels = train_test_split(images,
                                                                        labels_one_hot,
                                                                        test_size=0.2,
                                                                        random_state=42)

In [27]:
#aplanamos las imágenes
train_images = train_images.reshape((train_images.shape[0], img_size[0]*img_size[1]*3))
test_images = test_images.reshape((test_images.shape[0], img_size[0]*img_size[1]*3))

In [33]:
#construimos el modelo de red neuronal
#definimos que el modelo será secuencial (una capa después de la otra)
model = models.Sequential()
#agregamos las capas de la red
model.add(layers.Dense(128, activation = 'relu', input_shape = (img_size[0]*img_size[1]*3,)))
model.add(layers.Dense(2, activation = 'softmax'))


In [34]:
#compilamos el modelo
model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#entrenamos el modelo
history = model.fit(train_images, train_labels,
                    epochs = 20,
                    validation_data = (test_images, test_labels))

In [37]:
#evaluamos el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7368 - loss: 0.4973
Test accuracy: 0.7368420958518982
